# Functions and Decorators in Python

## Functions

Functions are python objects taking a set of arguments (or none), and performing a set of operations. They can be held un-executed.

Functions are first-class citizens in python, meaning they can be passed as arguments in other functions, returned by other functions, and even assigned to variables.

More info: https://www.youtube.com/watch?v=FsAPt_9Bf3U

(Thank you Corey Schafer)

### Example1: Functions can be assigned to variables

In [1]:
def square(x):
    return x*x

In [4]:
f = square
f

<function __main__.square(x)>

In this example, the variable f is set up equal to the function square (which is unexectuted, since there are no parenthesis). f is a function object equal to the square function object defined previously.

The variable f can now be executed in the same way as the function square, as shown below.

In [5]:
f(5)

25

### Example 2: functions can be inserted in arguments

In [6]:
def map_f (fun, array):
    result = []
    for i in array:
        result.append(fun(i))
    return result

In this case, a function is created accepting another function as an argument. We can use the function variable f defined previously, and an input array of [1,2,3,4] to return the squares of the vector.

In [7]:
map_f(f,[1,2,3,4])

[1, 4, 9, 16]

### Example 3: functions can be returned

In [21]:
def html_tag_msg(tag):
    def wrap_msg(msg):
        print ('<{0}>{1}</{0}>'.format(tag, msg))
    return wrap_msg

In this example, we define a function to create another function (which is returned). Calling ```html_tag_msg``` will create a ```wrap_msg``` function that will add the HTML ```tag``` specified. That function created can then be called to add the message we require.

In [24]:
h1 = html_tag_msg('h1')
h1

<function __main__.html_tag_msg.<locals>.wrap_msg(msg)>

In [25]:
h1('Heading')

<h1>Heading</h1>


Note that the function ```wrap_msg``` remembered that ```'h1'``` was inserted as an argument of the outer function ```htm_tag_msg```, and still has access to it. This is allowed by a concept called closures.

## Closures

Closures are the way Python 'remembers' inner variables that functions have access to.
"A closure closes over the free variables from their environment".

#### Example: creating a logger function

In [48]:
def my_logger(funct):
    import logging
    logging.basicConfig(filename = 'example.log',
                        level = logging.INFO)
    def log_funct(*args):
        logging.info('Running"{}" with arguments {}'.format(funct.__name__, args))
        print (funct(*args))
    return log_funct

def add(x, y):
    return x + y

add_logger = my_logger(add)

# add_logger(3, 4)

```python
def logger(funct):```
Defines a logger function that creates an inner ```log_funct``` logging (any) function in an 'example.log' file.

```python
def log_funct(*args):```
Defines a log_function accepting any arguments we input (thanks to the ```*args``` inserted) that will log the name of the funtion and the arguments inserted in the 'example.log' file, and will return the value of the sum in the console.

```python
add_logger(3,4)```
Will call the function ```log_funct```, which will remember its inner function ```funct``` is ```sum``` thanks to closures. It will compute the sum, and log it into the log file.

Note: ```funct.__name__``` is a method returning the name of the function inserted.

# Decorators

Decorators are used as functions defined to add functionalities to other functions.

Decorators are functions taking another function as an argument. They adding some kind of functionality to it, and then return another function. All of this without altering the source code of the original function passed in as argument.

They are usually used to create loggers for functions, or timing for how long functions run (examples at the end).

Note: decorators are used as class extenders as well.

### Example:

In [34]:
def decorator_fnct(original_fnct):
    def wrapper_fnct():
        print('Wrapper executed with {}'.format(original_fnct.__name__))
        return original_fnct()
    return wrapper_fnct

This first set of code defines a decorator function (decorator since it expects a function as an argument and returns another function), which it then uses to create a wrapper function returning a message and the original function executed.

In [35]:
@decorator_fnct
def display():
    print('Hello!')

What here the code does is 'extend' the code of the function ```display``` to add the functionalities defined in the decorator function.

```python
@decorator_fnct
#Is the same as:
display = decorator_fnct(display)```
Tells the program to set the function display equal to ```decorator_fnct(display)```. Now, exectuting display shoud add the functionalities of the decorator:

In [36]:
display()

Wrapper executed with display
Hello!


### Example 2: adding ```*args``` and ```**kwargs```

If we try using the previous ```decorator_fnct``` for a function requiring arguments, Python will give an error. 

To avoid that, we need to tell the decorator to accept any positional argument and any key-word argument inserted in the original function with ```*args``` and ```**kwargs``` statements.

#### Example of error:

In [39]:
@decorator_fnct
def display_info(name, age):
    print('My name is {} and I am {} years old'.format(name, age))
display_info('Manuel', '27')

TypeError: wrapper_fnct() takes 0 positional arguments but 2 were given

#### Correct statement:

In [40]:
def decorator_fnct(original_fnct):
    def wrapper_fnct(*args, **kwargs):
        print('Wrapper executed with {}'.format(original_fnct.__name__))
        return original_fnct(*args, **kwargs)
    return wrapper_fnct

@decorator_fnct
def display_info(name, age):
    print('My name is {} and I am {} years old'.format(name, age))
display_info('Manuel', '27')

### Syntax for class decorator (same functionality)

The same functionality can be implemented using a class decorator, which adds the same functionalities to functions.

In this case, a ```__call__``` method needs to be initialized to state the action taken when the instance of the ```decorator_class``` is called.

In [46]:
class decorator_class(object):
    def __init__(self, original_fnct):
        self.original_fnct = original_fnct
    
    def __call__(self, *args, **kwargs):
        print('Call method executed with {}'.format(self.original_fnct.__name__))
        return self.original_fnct(*args, **kwargs)

In [47]:
@decorator_class
def display_info(name, age):
    print('My name is {} and I am {} years old'.format(name, age))
display_info('Manuel', '27')

Call method executed with display_info
My name is Manuel and I am 27 years old


## Stacking decorators

When trying to stack many decorators at a time, we will find out that the first time our original function is called by the decorator, it will get the name of the wrapper function.

To avoid that, the ```functools``` package has got our back. We just need to import the ```wraps``` module and add it to all the wrapper functions, as shown below.

In [50]:
from functools import wraps

# Logger decorator
def my_logger(funct):
    import logging
    logging.basicConfig(filename = 'example.log',
                        level = logging.INFO)
    
    @wraps(funct) #ADD THIS to avoid 'funct' name to be changed to 'wrapper'
    def wrapper(*args, **kwargs):
        logging.info('Ran with args:"{}" and kwargs {}'.format(args, kwargs))
        print (funct(*args, **kwargs))
    return wrapper

#Timer decorator
def my_timer(funct):
    import time
    
    @wraps(funct) #ADD THIS to avoid 'funct' name to be changed to 'wrapper'
    def wrapper(*args, **kwargs):
        t1=time.time()
        resul = funct(*args, **kwargs)
        t2 = time.time() - t1
        print('{} ran in: {} sec'.format(funct.__name___, t2))
    return wrapper

Adding ```@wraps(funct)``` allows the function passed into a wrapper to keep its original name. Even if the decorator returns ```wrapper``` or any name, the module will make the function keep its name.